### Overfing

In [3]:
import pandas as pd

df = pd.read_parquet('/kaggle/input/cleaned-mutations-phd/df.parquet.hugo.snp.ins.del.all.qzip')

In [7]:
df = df.dropna(subset=['Gene', 'Exon'])

In [8]:
df

,primary_site,file_gdc_id,project_short_name,Variant_Type,Variant_Classification,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Gene,Feature,Hugo_Symbol,Exon,Start,End,Genomic_Bin,Strand,dataset,Chromosome,Mutational_Motif
0,Hematopoietic and reticuloendothelial systems,5c0a24b5-ed57-4ac8-b7c7-6b839622ca47,BEATAML1.0-COHORT,DEL,Frame_Shift_Del,GCGGCAGGTGGGGCGGGAGGCT,GCGGCAGGTGGGGCGGGAGGCT,-,ENSG00000245848,ENST00000498907,CEBPA,1/1,33302272,33302293,chr19_33000000,+,`isb-cgc-bq.BEATAML1_0.masked_somatic_mutation...,chr19,G[del]T
1,Hematopoietic and reticuloendothelial systems,ad0dda7c-8df4-4160-a1a7-6657a7c4b74b,BEATAML1.0-COHORT,DEL,In_Frame_Del,AGT,AGT,-,ENSG00000134480,ENST00000256897,CCNH,7/9,87399494,87399496,chr5_87000000,+,`isb-cgc-bq.BEATAML1_0.masked_somatic_mutation...,chr5,A[del]T
2,Hematopoietic and reticuloendothelial systems,ff14dfb7-a8ae-444a-8389-f189dfbb1f86,BEATAML1.0-COHORT,DEL,Frame_Shift_Del,CGTTCCA,CGTTCCA,-,ENSG00000108312,ENST00000302904,UBTF,13/21,44210812,44210818,chr17_44000000,+,`isb-cgc-bq.BEATAML1_0.masked_somatic_mutation...,chr17,T[del]A
4,Hematopoietic and reticuloendothelial systems,9be6cc7f-3f0c-49c8-b860-b333aa76dff3,BEATAML1.0-COHORT,DEL,Frame_Shift_Del,AC,AC,-,ENSG00000164754,ENST00000297338,RAD21,10/14,116852694,116852695,chr8_116000000,+,`isb-cgc-bq.BEATAML1_0.masked_somatic_mutation...,chr8,G[del]C
5,Hematopoietic and reticuloendothelial systems,95a5341f-4ad8-4d58-83d5-3cd5515ce210,BEATAML1.0-COHORT,DEL,Splice_Site,TGGGACATCTGC,TGGGACATCTGC,GCCCCAGGT,ENSG00000181555,ENST00000409792,SETD2,20/21,47017732,47017743,chr3_47000000,+,`isb-cgc-bq.BEATAML1_0.masked_somatic_mutation...,chr3,C[del]C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570295,Corpus uteri,c676f7e1-de13-4c16-b106-a611228881b3,TCGA-UCEC,SNP,Missense_Mutation,T,T,C,ENSG00000173085,ENST00000311469,COQ2,6/7,83267629,83267629,chr4_83000000,+,`isb-cgc-bq.TCGA.masked_somatic_mutation_hg38_...,chr4,G[T>C]T
3570296,Bronchus and lung,76dce8cc-38ff-4ddf-8272-0e405d82f63a,TCGA-LUAD,SNP,Missense_Mutation,T,T,C,ENSG00000157219,ENST00000287907,HTR5A,1/2,155071267,155071267,chr7_155000000,+,`isb-cgc-bq.TCGA.masked_somatic_mutation_hg38_...,chr7,C[T>C]T
3570297,Bronchus and lung,ebe634ff-b960-4474-8c6c-1451ad866f6b,TCGA-LUAD,SNP,Missense_Mutation,T,T,C,ENSG00000111077,ENST00000314250,TNS2,12/29,53057611,53057611,chr12_53000000,+,`isb-cgc-bq.TCGA.masked_somatic_mutation_hg38_...,chr12,A[T>C]T
3570298,Corpus uteri,a34c7bfa-9a11-4974-812f-dc682482d39b,TCGA-UCEC,SNP,Silent,T,T,C,ENSG00000197121,ENST00000354764,PGAP1,4/27,196912985,196912985,chr2_196000000,+,`isb-cgc-bq.TCGA.masked_somatic_mutation_hg38_...,chr2,A[T>C]T


In [9]:
top_sites = df['primary_site'].value_counts().nlargest(24).index

filtered_df = df[df['primary_site'].isin(top_sites)]

filtered_df['primary_site'] = filtered_df['primary_site'].astype('category').cat.codes


grouped_df = filtered_df.groupby('file_gdc_id').agg({
    'Mutational_Motif': list,
    'Genomic_Bin': list,
    'Hugo_Symbol': list,
    'Exon': list, 
    'Feature': list,
    'Variant_Classification': list, 
    'primary_site': 'first'
}).reset_index()


/tmp/ipykernel_36/4185034200.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['primary_site'] = filtered_df['primary_site'].astype('category').cat.codes


In [10]:
list_a = ['Mutational_Motif',
    'Genomic_Bin',
    'Hugo_Symbol',
    'Exon', 
    'Feature',
    'Variant_Classification']

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer


tokenizers = {}
for column in list_a:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(grouped_df[column])
    tokenizers[column] = tokenizer


In [16]:
max_len = int(grouped_df['Mutational_Motif'].apply(len).quantile(0.75))

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = {}
for key in tokenizers.keys():
    X[key] = pad_sequences(tokenizers[key].texts_to_sequences(grouped_df[key]), maxlen=max_len, padding='post')

In [32]:
tokenizers

{'Mutational_Motif': <keras.src.legacy.preprocessing.text.Tokenizer at 0x7bfa572d5db0>,
 'Genomic_Bin': <keras.src.legacy.preprocessing.text.Tokenizer at 0x7bfa56900b80>,
 'Hugo_Symbol': <keras.src.legacy.preprocessing.text.Tokenizer at 0x7bfa55895c00>,
 'Exon': <keras.src.legacy.preprocessing.text.Tokenizer at 0x7bfa56905b40>,
 'Feature': <keras.src.legacy.preprocessing.text.Tokenizer at 0x7bfa54247790>,
 'Variant_Classification': <keras.src.legacy.preprocessing.text.Tokenizer at 0x7bfb740f34c0>}

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Attention, Concatenate, GlobalAveragePooling1D, Lambda, Add
import numpy as np

vocab_size_motif = len(tokenizers['Mutational_Motif'].word_index) + 1
vocab_size_genomic_bin = len(tokenizers['Genomic_Bin'].word_index) + 1
vocab_hugo = len(tokenizers['Hugo_Symbol'].word_index) + 1

vocab_variant = len(tokenizers['Variant_Classification'].word_index) + 1
vocab_feature = len(tokenizers['Feature'].word_index) + 1
vocab_exon = len(tokenizers['Exon'].word_index) + 1

embedding_dim = 512
num_classes = len(grouped_df['primary_site'].unique())
max_len = int(grouped_df['Mutational_Motif'].apply(len).quantile(0.75))

def create_model(embedding_dim, num_classes):
    def positional_encoding(position, d_model):
        angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model))
        sines = np.sin(angle_rads[:, 0::2])
        cosines = np.cos(angle_rads[:, 1::2])
        pos_encoding = np.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    # Wejścia
    input_motif = Input(shape=(max_len,), name='Mutational_Motif')
    input_genomic_bin = Input(shape=(max_len,), name='Genomic_Bin')
    input_hugo = Input(shape=(max_len,), name='Hugo_Symbol')
    # Wejścia 2
    input_variant = Input(shape=(max_len,), name='Variant_Classification')
    input_feature = Input(shape=(max_len,), name='Feature')
    input_exon = Input(shape=(max_len,), name='Exon')
    
    
    positional_encoding_layer = positional_encoding(max_len, embedding_dim)
    
    # main part
    motif_embedding = Embedding(input_dim=vocab_size_motif, output_dim=embedding_dim)(input_motif)
    genomic_bin_embedding = Embedding(input_dim=vocab_size_genomic_bin, output_dim=embedding_dim)(input_genomic_bin)
    hugo_embedding = Embedding(input_dim=vocab_hugo, output_dim=embedding_dim)(input_hugo)
    # second part
    variant_embedding = Embedding(input_dim=vocab_variant, output_dim=embedding_dim)(input_variant)
    feature_embedding = Embedding(input_dim=vocab_feature, output_dim=embedding_dim)(input_feature)
    exon_embedding = Embedding(input_dim=vocab_exon, output_dim=embedding_dim)(input_exon)
    
    motif_embedding = Add()([Add()([motif_embedding, variant_embedding]), positional_encoding_layer])
    genomic_bin_embedding = Add()([Add()([genomic_bin_embedding, feature_embedding]), positional_encoding_layer])
    hugo_embedding = Add()([Add()([hugo_embedding, exon_embedding]), positional_encoding_layer])

    
    attention_layer = Attention()([motif_embedding, genomic_bin_embedding, hugo_embedding])

    concat_layer = Concatenate()([motif_embedding, genomic_bin_embedding, hugo_embedding, attention_layer])

    gap_layer = GlobalAveragePooling1D()(concat_layer)

    output = Dense(num_classes, activation='softmax', name='project_short_name')(gap_layer)

    model = Model(inputs=[input_motif, input_genomic_bin, input_hugo, input_variant, input_feature, input_exon], outputs=[output])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [57]:
num_classes = len(grouped_df['primary_site'].unique())

In [58]:
X_gene = X['Hugo_Symbol']
X_genomic_bin = X['Genomic_Bin']
X_motif = X['Mutational_Motif']

X_exon = X['Exon']
X_feature = X['Feature']
X_variant = X['Variant_Classification']

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate, Lambda, GlobalAveragePooling1D
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_scores = []
all_scores_top3 = []
all_scores_top5 = []
all_precisions = []
all_recalls = []

# Assuming y is already defined as:
y = np.array(grouped_df['primary_site'].tolist())

for train_index, test_index in kf.split(X_motif):
    X_train_gene, X_test_gene = X_gene[train_index], X_gene[test_index]
    X_train_genomic_bin, X_test_genomic_bin = X_genomic_bin[train_index], X_genomic_bin[test_index]
    X_train_motif, X_test_motif = X_motif[train_index], X_motif[test_index]

    X_train_exon, X_test_exon = X_exon[train_index], X_exon[test_index]
    X_train_feature, X_test_feature = X_feature[train_index], X_feature[test_index]
    X_train_variant, X_test_variant = X_variant[train_index], X_variant[test_index]
    
    
    y_train, y_test = y[train_index], y[test_index]

    model = create_model(embedding_dim, num_classes)

    model.fit([X_train_motif, X_train_genomic_bin, X_train_gene, X_train_variant, X_train_feature, X_train_exon], y_train, epochs=20, validation_split=0.1, batch_size=128, verbose=1)

    scores = model.evaluate([X_test_motif, X_test_genomic_bin, X_test_gene, X_test_variant, X_test_feature, X_test_exon], y_test, verbose=0)
    all_scores.append(scores[1])

    predictions = model.predict([X_test_motif, X_test_genomic_bin, X_test_gene, X_test_variant, X_test_feature, X_test_exon])

    y_pred_classes = np.argmax(predictions, axis=1)

    # Calculate precision and recall
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')

    all_precisions.append(precision)
    all_recalls.append(recall)

    top_3 = np.argsort(predictions, axis=1)[:, -3:]
    correct = np.sum([1 if y_test[i] in top_3[i] else 0 for i in range(len(y_test))])
    top_3_accuracy = correct / len(y_test)

    all_scores_top3.append(top_3_accuracy)

    top_5 = np.argsort(predictions, axis=1)[:, -5:]
    correct = np.sum([1 if y_test[i] in top_5[i] else 0 for i in range(len(y_test))])
    top_5_accuracy = correct / len(y_test)

    all_scores_top5.append(top_5_accuracy)

print(f'Srednia dokladnosc top1: {np.mean(all_scores)}')
print(f'Srednia dokladnosc top3: {np.mean(all_scores_top3)}')
print(f'Srednia dokladnosc top5: {np.mean(all_scores_top5)}')
print(f'Srednia precyzja: {np.mean(all_precisions)}')
print(f'Srednia recall: {np.mean(all_recalls)}')


Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 61s 622ms/step - accuracy: 0.3281 - loss: 2.4783 - val_accuracy: 0.4892 - val_loss: 1.9385
Epoch 2/20
30/95 ━━━━━━━━━━━━━━━━━━━━ 38s 600ms/step - accuracy: 0.4879 - loss: 1.9026